In [1]:
import os
import torch
import pandas as pd
from modules.feature_engineering import FeatureEngineeringV1
from modules.data_module import FlightDataModule
from modules.training import TrainingFlightModel
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

print(torch.__version__)
fe = FeatureEngineeringV1()

2.1.2+cu121


In [2]:
%load_ext tensorboard

In [3]:
def get_default_device():
	"""Pick GPU if available, else CPU"""
	if torch.cuda.is_available():
		print(f'Using device: {torch.cuda.get_device_name(0)}')
		return torch.device('cuda')
	elif torch.backends.mps.is_available():
		print('Using device: Apple ARM GPU')
		return torch.device('mps')
	else:
		print('Using device: CPU')
		return torch.device('cpu')

# Set the device
device = get_default_device()
print(f'Using device: {device}')

Using device: NVIDIA GeForce RTX 3060
Using device: cuda


In [4]:
# Used with GPUs having Tensor cores
torch.set_float32_matmul_precision('medium')

In [5]:
processed_df = pd.read_csv('../data/csv/processed.csv')
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Ac_kts,Time_step,Ac_Lat,Ac_Lon,Ac_feet,Ac_type_1,Ac_type_2,Phase_1,Phase_2,Phase_3
0,0.928702,0.495425,-0.634564,0.022136,-1.145864,2.972182,-2.250708,5.614973,-1.593272,2.974218,...,-2.268434,-1.526071,-1.407072,-0.227452,-1.853900,1.0,0.0,1.0,0.0,0.0
1,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,-2.183981,-1.515271,-1.408670,-0.245917,-1.791555,1.0,0.0,1.0,0.0,0.0
2,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,-1.940006,-1.504471,-1.409917,-0.266631,-1.762331,1.0,0.0,1.0,0.0,0.0
3,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,-1.696031,-1.495831,-1.408996,-0.284978,-1.744796,1.0,0.0,1.0,0.0,0.0
4,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,-1.470824,-1.487191,-1.405175,-0.300484,-1.709727,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60523,-0.413190,0.599672,0.090406,-0.943672,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,-2.390422,1.358068,1.422109,-1.357957,-1.764279,0.0,1.0,1.0,0.0,0.0
60524,-0.413190,0.599672,0.090406,-0.943672,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,-2.437340,1.367788,1.421052,-1.342096,-1.783762,0.0,1.0,1.0,0.0,0.0
60525,-0.413190,0.599672,0.090406,-0.948077,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,-2.493642,1.376428,1.420185,-1.328957,-1.799348,0.0,1.0,1.0,0.0,0.0
60526,-0.413190,0.599672,0.090406,-0.948077,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,-2.634397,1.394788,1.418559,-1.304101,-1.828572,0.0,1.0,1.0,0.0,0.0


In [6]:
X, y = fe.padding_features(processed_df)
print(f'X.shape: {X.shape}')
print(f'y.shape: {y.shape}')
print(f'X.dtype: {X.dtype}')
print(f'y.dtype: {y.dtype}')
print(f'X:\n{X}')
print(f'y:\n{y}')

X.shape: (352, 348, 42)
y.shape: (352, 348, 4)
X.dtype: float64
y.dtype: float64
X:
[[[ 9.28701650e-01  4.95424832e-01 -6.34563623e-01 ...  0.00000000e+00
    0.00000000e+00 -1.52607089e+00]
  [ 9.28701650e-01  4.95424832e-01 -6.34563623e-01 ...  0.00000000e+00
    0.00000000e+00 -1.51527089e+00]
  [ 9.28701650e-01  4.95424832e-01 -6.34563623e-01 ...  0.00000000e+00
    0.00000000e+00 -1.50447089e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 1.34159134e+00  4.82187043e-01 -2.20295522e-01 ...  0.00000000e+00
    0.00000000e+00 -1.52283089e+00]
  [ 1.34159134e+00  4.82187043e-01 -2.20295522e-01 ...  0.00000000e+00
    0.00000000e+00 -1.51419089e+00]
  [ 1.34159134e+00  4.82187043e-01 -2.20295522e-01 .

In [7]:
X_train, X_val, X_test, y_train, y_val, y_test = fe.split_train_test(X, y, test_size=0.2, val_size=0.25)
print(f'Shape of training set is {X_train.shape}, {y_train.shape}')
print(f'Shape of validation set is {X_val.shape}, {y_val.shape}')
print(f'Shape of test set is {X_test.shape}, {y_test.shape}')

Shape of training set is (281, 348, 42), (281, 348, 4)
Shape of validation set is (53, 348, 42), (53, 348, 4)
Shape of test set is (18, 348, 42), (18, 348, 4)


In [8]:
train_dataset, val_dataset, test_dataset = fe.make_tensor_dataset(X_train, X_val, X_test, y_train, y_val, y_test)
print(f'training set is {train_dataset}')
print(f'validation set is {val_dataset}')
print(f'test set is {test_dataset}')

training set is <torch.utils.data.dataset.TensorDataset object at 0x7f743e0a3550>
validation set is <torch.utils.data.dataset.TensorDataset object at 0x7f743e61ec90>
test set is <torch.utils.data.dataset.TensorDataset object at 0x7f743e0a4c10>


# TRAINING

---

In [9]:
# Global variables
batch_size_variable = 16
max_epochs = 200

In [10]:
flight_dm = FlightDataModule(train_dataset, val_dataset, test_dataset, batch_size=batch_size_variable)

In [11]:
model_name = 'flight_model_LSTM_V2'
base_dir = os.getcwd().replace('notebook', '')
log_dir = os.path.join(base_dir, 'log')
ckpt_dir = os.path.join(base_dir, model_name)
os.makedirs(log_dir, exist_ok=True)

In [12]:
log_dir

'/mnt/e/Work/chi_phuong_co_su/log'

In [13]:
# Some necessary callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=True, mode='min')
model_checkpoint = ModelCheckpoint(
	dirpath=log_dir,
	filename='LSTM-V2-{epoch}-{val_loss:.5f}',
	monitor='val_loss',
	save_top_k=3,
	verbose=True,
	mode='min'
)
logger = TensorBoardLogger(log_dir, name=model_name, log_graph=True)

In [14]:
training_module = TrainingFlightModel(
	early_stopping_lstm = early_stopping,
	model_checkpoint_lstm = model_checkpoint,
	logger_lstm = logger,
	data_module = flight_dm,
	max_epoch=max_epochs,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
training_module.training(
	input_size=42,
	hidden_size=500,
	num_layers=10,
	output_size=4,
	learning_rate=1e-4,
	dropout=0.2,
)

Missing logger folder: /mnt/e/Work/chi_phuong_co_su/log/flight_model_LSTM_V2
/home/anvutrong/miniconda3/envs/wsl_env/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /mnt/e/Work/chi_phuong_co_su/log exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type              | Params | In sizes                                      | Out sizes                                     
--------------------------------------------------------------------------------------------------------------------------------------------
0 | lstm | LSTM              | 19.1 M | [[1, 6616, 42], [[10, 1, 500], [10, 1, 500]]] | [[1, 6616, 500], [[10, 1, 500], [10, 1, 500]]]
1 | fc   | Linear            | 2.0 K  | [1, 6616, 500]                                | [1, 6616, 4]                                  
2 | relu | ReLU              | 0      | [1, 6616, 500]                                | [1, 6616, 500]                               

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.477
Epoch 0, global step 18: 'val_loss' reached 0.47740 (best 0.47740), saving model to '/mnt/e/Work/chi_phuong_co_su/log/LSTM-V2-epoch=0-val_loss=0.47740.ckpt' as top 3


In [ ]:
training_module.validate_and_testing_lstm(data_module=flight_dm)

In [ ]:
# Tensorboard
%tensorboard --logdir /mnt/e/Work/chi_phuong_co_su/log